 Data contained in pandas objects can be combined together in a number of built-in
 ways:
 • pandas.merge connects rows in DataFrames based on one or more keys. This will
 be familiar to users of SQL or other relational databases, as it implements database
 join operations.
 • pandas.concat glues or stacks together objects along an axis.
 • combine_first instance method enables splicing together overlapping data to    fill in missing values in one object with values from another.

In [2]:

# Database-style DataFrame Merges
# Merge or join operations combine data sets by linking rows using one or more keys.
#  These operations are central to relational databases. The merge function in pandas is
#  the main entry point for using these algorithms on your data

import pandas as pd
import numpy as np

df1=pd.DataFrame({
    'key':['b','b','a','c','a','a','b'],
    'data1':range(7)
})

df2=pd.DataFrame({
    'key':['a','b','d'],
    'data2':range(3)
})

print(df1)
print('')
print(df2)

  key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   a      5
6   b      6

  key  data2
0   a      0
1   b      1
2   d      2


In [3]:
# This is an example of a many-to-one merge situation; the data in df1 has multiple rows
#  labeled a and b, whereas df2 has only one row for each value in the key column.

pd.merge(df1,df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [6]:
# Note that I didn’t specify which column to join on. If not specified, merge uses the
#  overlapping column names as the keys. It’s a good practice to specify explicitly, though

pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [9]:
#  If the column names are different in each object, you can specify them separately

df3 = pd.DataFrame({
     'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
      'data1': range(7)
})

df4 = pd.DataFrame({
    'rkey': ['a', 'b', 'd'],
    'data2': range(3)
})

pd.merge(df3,df4,left_on='lkey',right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [10]:
# The outer join takes the union of the keys, combining the effect of applying both left and right joins

pd.merge(df1,df2,how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [11]:
#  Many-to-many merges have well-defined though not necessarily intuitive behavior.
#  Here’s an example

df1 = pd.DataFrame({
    'key': ['b', 'b', 'a', 'c', 'a', 'b'],
    'data1': range(6)
})

df2 = pd.DataFrame({
     'key': ['a', 'b', 'a', 'b', 'd'],
     'data2': range(5)
 })
        
print(df1)        
print("")
print(df2)        
print('')
pd.merge(df1,df2, on='key',how='left')

# Many-to-many joins form the Cartesian product of the rows. Since there were 3 'b'
#  rows in the left DataFrame and 2 in the right one, there are 6 'b' rows in the result.
#  The join method only affects the distinct key values appearing in the result

  key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   b      5

  key  data2
0   a      0
1   b      1
2   a      2
3   b      3
4   d      4



,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [12]:
pd.merge(df1,df2,how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [14]:
# To merge with multiple keys, pass a list of column names

left = pd.DataFrame({
    'key1': ['foo', 'foo', 'bar'],
    'key2': ['one', 'two', 'one'],
    'lval': [1, 2, 3]
})

right = pd.DataFrame({
    'key1': ['foo', 'foo', 'bar', 'bar'],
    'key2': ['one', 'one', 'one', 'two'],
    'rval': [4, 5, 6, 7]
})

pd.merge(left,right,on=['key1','key2'],how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [15]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [16]:
 pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [ ]:
#  Table 7-1. merge function arguments// page=181

In [3]:
# Merging on Index

# In some cases, the merge key or keys in a DataFrame will be found in its index. In this
#  case, you can pass left_index=True or right_index=True (or both) to indicate that the
#  index should be used as the merge key

import pandas as pd

left1=pd.DataFrame({
    'key':['a', 'b', 'a', 'a', 'b', 'c'],
    'value':range(6)
})

right1=pd.DataFrame({
    'group_val':[3.5,7],
    'index':['a','b']
})

pd.merge(left1,right1,left_on='key',right_index=True)

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [4]:
 pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [6]:
import numpy as np

lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
                    'key2': [2000, 2001, 2002, 2001, 2002],
                    'data': np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                    index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                          [2001, 2000, 2000, 2000, 2001, 2002]],
                   columns=['event1', 'event2'])

print(lefth)
print(righth)

     key1  key2  data
0    Ohio  2000   0.0
1    Ohio  2001   1.0
2    Ohio  2002   2.0
3  Nevada  2001   3.0
4  Nevada  2002   4.0
             event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11


In [8]:
#  In this case, you have to indicate multiple columns to merge on as a list (pay attention
#  to the handling of duplicate index values)

pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [11]:
# Using the indexes of both sides of the merge is also not an issue

left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]], index=['a', 'c', 'e'],
                  columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                    index=['b', 'c', 'd', 'e'], columns=['Missouri', 'Alabama'])

print(left2)
print(right2)

   Ohio  Nevada
a   1.0     2.0
c   3.0     4.0
e   5.0     6.0
   Missouri  Alabama
b       7.0      8.0
c       9.0     10.0
d      11.0     12.0
e      13.0     14.0


In [12]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [13]:
# DataFrame has a more convenient join instance for merging by index

left2.join(right2,how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [14]:
left1.join(right1, on='key')

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [16]:
# Lastly, for simple index-on-index merges, you can pass a list of DataFrames to join as
#  an alternative to using the more general concat function described below

another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],index=['a', 'c', 'e', 'f'], columns=['New York', 'Oregon'])
left2.join([right2,another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [17]:
 left2.join([right2, another], how='outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0
b,NaN,NaN,7.0,8.0,NaN,NaN
d,NaN,NaN,11.0,12.0,NaN,NaN
f,NaN,NaN,NaN,NaN,16.0,17.0


In [1]:
# Concatenating Along an Axis

#  Another kind of data combination operation is alternatively referred to as concatena
# tion, binding, or stacking. NumPy has a concatenate function for doing this with raw
#  NumPy arrays

import numpy as np

arr=np.arange(12).reshape((3,4))
print(arr)

np.concatenate([arr,arr],axis=1)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]


array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [2]:
np.concatenate([arr,arr],axis=0)

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [3]:
#  In the context of pandas objects such as Series and DataFrame, having labeled axes
#  enable you to further generalize array concatenation

# The concat function in pandas provides a consistent way to address each of these con
# cerns. I’ll give a number of examples to illustrate how it works

import pandas as pd
s1=pd.Series([0,1],index=['a','b'])
s2=pd.Series([2,3,4],index=['c','d','e'])
s3=pd.Series([5,6],index=['f','g'])
# Calling concat with these object in a list glues together the values and indexes
pd.concat([s1,s2,s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [4]:
# By default concat works along axis=0, producing another Series. If you pass axis=1, the
#  result will instead be a DataFrame (axis=1 is the columns)
pd.concat([s1,s2,s3],axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [8]:
#  In this case there is no overlap on the other axis, which as you can see is the sorted
#  union (the 'outer' join) of the indexes. You can instead intersect them by passing
#  join='inner

s4=pd.concat([s1*5,s3])
print(pd.concat([s1,s4],axis=1))
pd.concat([s1,s4],axis=1,join='inner')

     0  1
a  0.0  0
b  1.0  5
f  NaN  5
g  NaN  6


,0,1
a,0,0
b,1,5


In [11]:
# You can even specify the axes to be used on the other axes with join_axes
pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']]) #not working 

TypeError: concat() got an unexpected keyword argument 'join_axes'

In [14]:
#  One issue is that the concatenated pieces are not identifiable in the result. Suppose
#  instead you wanted to create a hierarchical index on the concatenation axis. To do this,
#  use the keys argument
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [17]:
#  In the case of combining Series along axis=1, the keys become the DataFrame column
#  headers:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In [18]:
# The same logic extends to DataFrame objects
df1=pd.DataFrame(np.arange(6).reshape(3,2),index=['a','b','c'],columns=['one','two'])
df2=pd.DataFrame(5+np.arange(4).reshape(2,2),index=['a','c'],columns=['three','four'])
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [20]:
#  If you pass a dict of objects instead of a list, the dict’s keys will be used for the keys
pd.concat({'level1': df1, 'level2': df2}, axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [22]:
#  There are a couple of additional arguments governing how the hierarchical index is created
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],names=['upper', 'lower'])
    

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [24]:
# A last consideration concerns DataFrames in which the row index is not meaningful in
#  the context of the analysis

df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
print(df1)
print(df2)

          a         b         c         d
0 -0.618937 -0.547799 -0.643285  0.018901
1 -1.227756  0.545279  0.557790 -0.495149
2 -0.044881  0.707811  0.605238 -0.426418
          b         d         a
0 -0.623927 -1.531128  0.591949
1 -1.151752 -0.229380  0.748093


In [25]:
# In this case, you can pass ignore_index=True
pd.concat([df1, df2], ignore_index=True)

#  Table 7-2. concat function arguments page: 188

,a,b,c,d
0,-0.618937,-0.547799,-0.643285,0.018901
1,-1.227756,0.545279,0.557790,-0.495149
2,-0.044881,0.707811,0.605238,-0.426418
3,0.591949,-0.623927,NaN,-1.531128
4,0.748093,-1.151752,NaN,-0.229380


In [2]:
# Combining Data with Overlap

# Another data combination situation can’t be expressed as either a merge or concate
# nation operation. You may have two datasets whose indexes overlap in full or part. As
#  a motivating example, consider NumPy’s 'where' function, which expressed a vectorized
#  if-else

import numpy as np
import pandas as pd

a=pd.Series([np.nan,2.5,np.nan,3.5,4.5,np.nan],index=['f', 'e', 'd', 'c', 'b', 'a'])
b=pd.Series(np.arange(len(a)),dtype=np.float64,index=['f', 'e', 'd', 'c', 'b', 'a'])
print(a)
print(b)
np.where(pd.isnull(a),b,a)

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64
f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    5.0
dtype: float64


array([0. , 2.5, 2. , 3.5, 4.5, 5. ])

In [4]:
# With DataFrames, combine_first naturally does the same thing column by column, so
#  you can think of it as “patching” missing data in the calling object with data from the
#  object you pass

df1=pd.DataFrame({
    'a':[1,np.nan,5,np.nan],
    'b':[np.nan,2,np.nan,6],
    'c':range(2,18,4)
})
df2=pd.DataFrame({
    'a':[5., 4., np.nan, 3., 7.],
    'b':[np.nan, 3., 4., 6., 8.]
})

df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


In [ ]:
# DONE